<a href="https://colab.research.google.com/github/hackerpranavpandey/SOI_Mission_Mars_Terrain/blob/main/Test_ResNet152.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path='/content/drive/MyDrive/Mission_Mars'
os.listdir(path)

['train.csv', 'train_dataset', 'test_dataset']

In [ ]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
images=[]
image_filenames=[]
def image_process(images_path):
    for image_path in os.listdir(images_path):
        image_name = os.path.join(images_path, image_path)
        image_array = cv2.imread(image_name, cv2.IMREAD_COLOR)
        image_array = cv2.resize(image_array, (224, 224))
        image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
        images.append(image_array)
        image_filenames.append(image_path)
image_process('/content/drive/MyDrive/Mission_Mars/test_dataset')

In [ ]:
import torch
from torch.utils.data import Dataset
from PIL import Image
class CustomDataset(Dataset):
    def __init__(self,images,transform=None):
        self.images = images
        self.transform = transform
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        image = self.images[idx]
        image = Image.fromarray(image)
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
from torchvision import transforms
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                     ])

In [ ]:
from torch.utils.data import DataLoader
test_dataset = CustomDataset(images, transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
import torch
import torchvision.models as models
from torch import nn
num_classes = 8
model = models.resnet152(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model_path = '/content/drive/MyDrive/model.pth'
model.load_state_dict(torch.load(model_path))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:01<00:00, 154MB/s]


<All keys matched successfully>

In [ ]:
def get_predictions(model, test_loader):
    model.eval()
    all_outputs = []
    with torch.no_grad():
        for inputs in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_outputs.extend(predicted.cpu().numpy())
    return all_outputs

In [ ]:
pred_labels = get_predictions(model, test_loader)
import pandas as pd
results = pd.DataFrame({
    'Filename': image_filenames,
    'Prediction': pred_labels
})
results.to_csv('/content/drive/MyDrive/Mission_Mars/pred_resnet152.csv', index=False)
print("Predictions saved to CSV file.")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Predictions saved to CSV file.
